### Индивидуальное задание.

#### Построить графики функций f(x) и g(x), уточнить координаты точек пересечения, решая численно соответствующее уравнение. На графике отметить и подписать буквами A1, A2, ... точки пересечения графиков, вывести в легенде формулы функций, подписать оси.

In [8]:
import numpy as np
import sympy as sp
from sympy.abc import x
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
%run document_generator.ipynb

In [16]:
"""
Returns two "nearby placed" functions and two points, at which each function intersect other.
"""
def generate_functions() -> tuple[sp.Function, sp.Function, sp.Point2D, sp.Point2D]:
    def coef(hi, lo = 1):
        return sp.sympify(np.random.choice([-1, 1]) * np.random.randint(lo, hi + 1))

    while True:
        a = coef(3) / coef(2)
        b = coef(8, 0) / coef(3)
        f = a*x**2 + b*x + coef(12, 0) / coef(4)
        
        x_mid = int(-b / (2 * a))
        x1 = x_mid - np.random.randint(1, 5) 
        x2 = x_mid + np.random.randint(1, 5)

        y1 = f.subs({x: x1})
        y2 = f.subs({x: x2})

        # Disallow similar heights
        if y1 - y2 < 1:
            continue

        # Disallow points that are too far 
        if sp.sqrt((x1 - x2)**2 + (y1 - y2)**2) > sp.S(20):
            continue

        solutions = sp.linsolve((sp.Matrix([[x1**2, x1, 1], [x2**2, x2, 1]]), sp.Matrix([y1, y2]))).args
        assert len(solutions) > 0, "No solutions found"

        a, b, c = solutions[0]
        free = c if isinstance(c, sp.Symbol) else (b if b != isinstance(b, sp.Symbol) else a)

        substitute = np.random.randint(-10, 11)
        a = a.subs({free: substitute})
        if a == 0:
            continue 

        b = b.subs({free: substitute})
        c = c.subs({free: substitute})

        g = a*x**2 + b*x + c
        if f == g:
            continue

        f, g = np.random.permutation([f, g])

        if np.random.randint(2) == 0:
            f = sp.factor(f)

        if np.random.randint(2) == 0:
            g = sp.factor(g)

        return f, g, sp.Point2D(x1, y1), sp.Point(x2, y2)

In [11]:
PICS_PATH = "../out/answers/answer5/"

In [12]:
def create_plot(f: sp.Function, 
                g: sp.Function,
                p1: sp.Point2D,
                p2: sp.Point2D, 
                name: str):
    xmin = float(min(p1[0], p2[0]) - 3)
    xmax = float(max(p1[0], p2[0]) + 3)
    ymin = float(min(p1[1], p2[1]) - 3)
    ymax = float(max(p1[1], p2[1]) + 3)

    p1_f = (float(p1[0]), float(p1[1]))
    p2_f = (float(p2[0]), float(p2[1]))

    X = np.linspace(xmin, xmax, 1000)
    
    lam_f = sp.lambdify(x, f, modules=["numpy"])
    lam_g = sp.lambdify(x, g, modules=["numpy"])

    f_vals = lam_f(X)
    g_vals = lam_g(X)

    fig = plt.figure()

    plt.plot(X, f_vals, label=f"$f(x) = {sp.latex(f)}$")
    plt.plot(X, g_vals, label=f"$g(x) = {sp.latex(g)}$")

    plt.scatter(*p1_f, color="black")
    plt.scatter(*p2_f, color="black")

    plt.annotate('A1', p1_f, xytext=(p1_f[0] + 0.8, p1_f[1] + 0.8), arrowprops={ "arrowstyle": '-'})
    plt.annotate('A2', p2_f, xytext=(p2_f[0] + 0.8, p2_f[1] + 0.8), arrowprops={ "arrowstyle": '-'})

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.xlabel("x")
    plt.ylabel("y")

    plt.legend()

    os.makedirs(os.path.dirname(PICS_PATH), exist_ok=True)
    
    plt.savefig(PICS_PATH + name)
    
    # Close plot to save on resources.
    plt.close(fig)

In [19]:
DESCRIPTION = """Построить графики функций $f(x)$ и $g(x)$, уточнить координаты точек пересечения, решая численно соответствующее уравнение. 
На графике отметить и подписать буквами A1, A2, ... точки пересечения графиков, вывести в легенде формулы функций, подписать оси."""

TASK = """\\begin{{align*}}
    f(x) = {f} && g(x) = {g}
\\end{{align*}}"""

SOLUTION = """\\begin{{align*}}
    f(x) = {f} && g(x) = {g}
\\end{{align*}}
Point 1: ${p1}$
Point 2: ${p2}$

\\includegraphics[scale=0.8]{{ {{ {plot_name} }} }}
"""

In [14]:
def variant_generator(variant_n: int):
    f, g, p1, p2 = generate_functions() 

    name = f"{variant_n}.png"
    create_plot(f, g, p1, p2, name)

    f_latex = sp.latex(f)
    g_latex = sp.latex(g)

    return TASK.format(f=f_latex, g=g_latex), SOLUTION.format(f=f_latex, g=g_latex, p1=sp.latex(p1), p2=sp.latex(p2), plot_name=name)

DOC = DocumentGenerator(variant_generator, DESCRIPTION)

In [20]:
write_tasks_and_solutions(DOC, "../out/tasks/task-5.tex", "../out/answers/answer5/answer-5.tex", 150)